<a href="https://colab.research.google.com/github/Or-Gindes/MachineLearningAssignment3/blob/master/Assignment3_part2_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PreProcessing**

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=89126523b550d89c7ffdf5c09514421a963d8b276c7ce9af2264f2dd151a5a25
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import tensorflow
from tensorflow.keras.applications import VGG19, vgg19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tarfile
import scipy.io
import tarfile
import os
from urllib.request import urlretrieve
import tarfile
import io
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt

seed = 123

In [ ]:
# Get images & labels and extract in current folder
dataset = "https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz"
labels = "https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat"
urlretrieve(dataset, dataset.rsplit('/', 1)[-1])
urlretrieve(labels, labels.rsplit('/', 1)[-1])

tgz_file = dataset.rsplit('/', 1)[-1]
with tarfile.open(tgz_file, 'r:gz') as file:
    # Extract all files to the specified directory
    file.extractall('.')

In [ ]:
# Load image labels into y and count unique classes
mat = scipy.io.loadmat(labels.rsplit('/', 1)[-1])
y = pd.Series(mat['labels'][0])
unique_labels = y.unique()
print(f"Number of unique classes: {len(unique_labels)}")

Number of unique classes: 102


In [ ]:
# Prepare dataframe for the ImageDataGenerator.flow_from_dataframe pipeline
data_dir = os.path.join(os.getcwd(), 'jpg')
print(f"data directory: {data_dir}")
df = pd.DataFrame({"filename": sorted(os.listdir(data_dir)),  "class": y.astype(str)})#, 'label': to_categorical(y-1).tolist()})
df.head()

data directory: /content/jpg


,filename,class
0,image_00001.jpg,77
1,image_00002.jpg,77
2,image_00003.jpg,77
3,image_00004.jpg,77
4,image_00005.jpg,77


In [ ]:
def get_train_val_test_split(df, label_col='class', seed=123):
  """
  Split DataFrame df to train/validation/test dataframes with 50%/25%/25% ratio
  """
  # Divide dataset randomly into train, validation, and test sets
  train_df, val_test_df = train_test_split(df, stratify=df[label_col],
                                          test_size=0.5,
                                          random_state=seed,
                                          shuffle=True)

  val_df, test_df = train_test_split(val_test_df,
                                    stratify=val_test_df[label_col],
                                    test_size=0.5,
                                    random_state=seed,
                                    shuffle=True)

  return train_df, val_df, test_df

In [ ]:
from random import random

def add_noise_with_func(img):
    '''Add random noise to an image'''
    VARIABILITY = 50
    deviation = VARIABILITY*random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    img = vgg19.preprocess_input(img)
    return img

def get_data_generators(df, data_dir, preprocess_func, x_col='filename', y_col='class',
                        input_shape=(224, 224), batch_size=32, seed=123):
  """
  Accepts dataframe of image file names, folder with images, preprocessing function
  and other optional parameters and returns train/validation/test data generators
  """
  train_df, val_df, test_df = get_train_val_test_split(df, y_col, seed)

  # Define image preprocessing and augmentation options
  datagen = ImageDataGenerator(
      preprocessing_function=preprocess_func
      # shear_range=0.2,
      # zoom_range=0.2,
      # horizontal_flip=True
  )

  # Define Image generator for each of train/validation/test set
  print("Training set:")
  train_generator = datagen.flow_from_dataframe(
      train_df,
      directory=data_dir,
      x_col=x_col,
      y_col=y_col,
      target_size=input_shape,
      batch_size=batch_size,
      class_mode="categorical",
      shuffle=True,
      seed=seed
  )
  # Batch size isn't needed for val and test generators because we want to
  # evaluated score for entire set
  print("Validation set:")
  val_generator = datagen.flow_from_dataframe(
      val_df,
      directory=data_dir,
      x_col=x_col,
      y_col=y_col,
      target_size=input_shape,
      class_mode="categorical",
      shuffle=False,
      seed=seed
  )
  print("Test set:")
  test_generator = datagen.flow_from_dataframe(
      test_df,
      directory=data_dir,
      x_col=x_col,
      y_col=y_col,
      target_size=input_shape,
      class_mode="categorical",
      shuffle=False,
      seed=seed
  )
  return train_generator, val_generator, test_generator

In [ ]:
def get_base_model():
  # Define base model from VGG10 pre-trained on imagenet
  base_model = VGG19(weights="imagenet", include_top=False,
                     input_shape=(224, 224, 3))
  return base_model

def get_top_model(y):
  # Define a new top model - output layer should have nodes same as number of labels in y
  n_classes = len(y.unique())

  top_model = Sequential()
  top_model.add(GlobalAveragePooling2D())
  top_model.add(Dense(n_classes, activation='softmax'))

  # top_model = Sequential()
  # top_model.add(Dropout(0.2))
  # top_model.add(GlobalAveragePooling2D())
  # top_model.add(Dense(n_classes, activation='softmax'))
  return top_model

def train_and_evaluate(df, data_dir, preprocess_func, model_name, config,
                       epochs=20, n=2):
  """
  Compile a model from a vgg19 base model and a top model
  Train the composite model with given optimizer for given number of epochs.
  Evaluate the model and repeat n times to generate accurate evaluation
  Log results in wandb
  """
  base_seed = 42
  seed_gen = iter(range(base_seed, base_seed*n + 1, base_seed))
  histories = []
  test_results = {'Accuracy': [], 'Loss': []}
  for i in range(n):
    print(f"\nTraining-Evaluation cycle - {i+1} - Model: {model_name}")

    # Get an instance of base and top models
    base_model = get_base_model()
    top_model = get_top_model(df['class'])

    # Get the data generators
    train_generator, validation_generator, test_generator = get_data_generators(df, data_dir, preprocess_func, seed=next(seed_gen))

    # Freeze base_model layers
    base_model.trainable = False

    # Combine the base_model with new top model
    model = Sequential()
    model.add(base_model)
    model.add(top_model)

    model.compile(optimizer=tensorflow.keras.optimizers.RMSprop(weight_decay=0.0001, centered=True),
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True)

    print("\nTraining:")
    history = model.fit(train_generator, epochs=epochs, callbacks=[early_stopping],
                        validation_data=validation_generator, verbose=1)
    histories.append(history)

    # Evaluate the model on the test set
    print("\nEvaluating:")
    loss, accuracy = model.evaluate(test_generator)
    test_results['Accuracy'].append(accuracy)
    test_results['Loss'].append(loss)

    print("Test Loss:", round(loss, 3))
    print(f"Test Accuracy: {round(accuracy * 100, 3)}%")

  evaluate(model_name, epochs, test_results, histories, config)
  return

def element_wise_avg(nested_lists):
  return [sum(elements) / len(elements) for elements in zip(*nested_lists)]

def evaluate(name, epochs, test_results, histories, config):
  wandb.init(project='MachineLearning_Assignment3',
            name=name,
            config=config)

  acc =  round(np.mean(test_results["Accuracy"]), 3)
  loss = round(np.mean(test_results["Loss"]), 3)
  wandb.log({"test_accuracy": acc, "test_loss": loss})

  avg_loss = element_wise_avg([history.history["loss"] for history in histories])
  avg_val_loss = element_wise_avg([history.history["val_loss"] for history in histories])
  avg_accuracy = element_wise_avg([history.history["accuracy"] for history in histories])
  avg_val_accuracy = element_wise_avg([history.history["val_accuracy"] for history in histories])
  wandb.log({"train_accuracy": np.max(avg_accuracy), "train_loss": np.min(avg_loss)})
  wandb.log({"validation_accuracy": np.max(avg_val_accuracy), "validation_loss": np.min(avg_val_loss)})
  wandb.log({"epochs": max([history.params['epochs'] for history in histories])})

  fig, axs = plt.subplots(figsize=(10, 5), ncols=2)
  axs = axs.ravel()
  axs[0].plot(avg_loss, label="Training")
  axs[0].plot(avg_val_loss, label="Validation")
  axs[0].set_title(f"{name}_Loss")
  axs[0].legend(loc='upper right')

  axs[1].plot(avg_accuracy, label="Training")
  axs[1].plot(avg_val_accuracy, label="Validation")
  axs[1].set_title(f"{name}_Accuracy")
  axs[1].legend(loc='lower right')

  fig.show()
  wandb.log({f"{name}_accuracy_loss": fig})

  wandb.finish()


# **VGG19**

Preprocessing for this model is accomplished with the preprocess_input function which converts the input images from RGB to BGR and zero-centers each color channel with respect to the ImageNet dataset, without scaling the image. The images are read with target_size 224x224 which is the input side of the model

In [ ]:
# Load the pre-trained VGG19 model without the top (classification) layers
# to observe the structure of the model
vgg_base_model = VGG19(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
vgg_base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In order to tune the hyperparameters of the model its required to repeat the splitting process of the dataset a number of times, training and evaluating the model with each split to get an accurate evaluation

In [ ]:
epochs = 50
config = {"base_model": "VGG19", "top_model": "AvgPooling+Dense",
                    "Preprocessing": "vgg19_preprocess_input",
                    "optimizer": "RMSprop_tuned", "Epochs": epochs}

train_and_evaluate(df, data_dir, model_name="VGG19_RMSProp_tuned", config=config,
                   preprocess_func=vgg19.preprocess_input, epochs=epochs, n=2)


Training-Evaluation cycle - 1 - Model: VGG19_RMSProp_tuned
Training set:
Found 4094 validated image filenames belonging to 102 classes.
Validation set:
Found 2047 validated image filenames belonging to 102 classes.
Test set:
Found 2048 validated image filenames belonging to 102 classes.

Training:
Epoch 1/50
128/128 [==============================] - 49s 373ms/step - loss: 5.1678 - accuracy: 0.2157 - val_loss: 2.3915 - val_accuracy: 0.4651
Epoch 2/50
128/128 [==============================] - 48s 379ms/step - loss: 1.4300 - accuracy: 0.6448 - val_loss: 1.2830 - val_accuracy: 0.6771
Epoch 3/50
128/128 [==============================] - 48s 376ms/step - loss: 0.6644 - accuracy: 0.8149 - val_loss: 0.9518 - val_accuracy: 0.7592
Epoch 4/50
128/128 [==============================] - 48s 376ms/step - loss: 0.3718 - accuracy: 0.9006 - val_loss: 0.7862 - val_accuracy: 0.7909
Epoch 5/50
128/128 [==============================] - 49s 384ms/step - loss: 0.2314 - accuracy: 0.9387 - val_loss: 0.746